## How to create a stochastic model for MNIST? (I.e., a model which can be used for MC-Dropout and Point Predictions)

In [ ]:
import tensorflow as tf

try:
  import uncertainty_wizard as uwiz
except ModuleNotFoundError as e:
  # Uncertainty wizard was not installed. Install it now (we're probably on colab)
  !pip install uncertainty_wizard
  import uncertainty_wizard as uwiz

**Step 1: Downloading Preprocessing the data**

This is the same that we would do on any regular keras mnist classifier,
except that we do not have to one-hot encode the test labels, as uncertainty wizards quantifiers
will determine the winning class (one not its one-hot encoding) for us

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype('float32') / 255).reshape(x_train.shape[0], 28, 28, 1)
x_test = (x_test.astype('float32') / 255).reshape(x_test.shape[0], 28, 28, 1)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)


**Step 2: Creating a Stochastic Uncertainty-Wizard Model using the Sequential API**

Note that only the first line is different from doing the same in plain keras, the rest is equivalent.
We must however ensure to use at least one randomized layer (e.g. tf.keras.layers.Dropout)

In [ ]:
# Let's create an uncertainty wizard model!
#   We want to use a dropout-based stochastic model using the functional interface

model = uwiz.models.StochasticSequential()

In [ ]:
# In the functional interface, we can add the layer as if we were working
#   on a regular tf.keras.model.Sequential() using add()

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Because we are building a stochastic model, we have to include at last one of the following keras layers
#   tf.keras.layers.Dropout, tf.keras.layers.GaussionNoise, tf.keras.layers.GaussianDropout
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
# Compiling and fitting is the same as in regular keras models as well:

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Note that we set the number of epochs to just 1, to be able to run this notebook quickly
# Set the number of epochs higher if you want to optimally train the network.
# Also note that obviously, with epochs=1, the currently passed callback does not do anything.
model.fit(x_train, y_train, validation_split=0.1, batch_size=32, epochs=1,
                  verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])


In [ ]:
# Let's print the summary of the inner model that is wrapped by the stochastic model we just created.
#   Note that the Dropout-Layer was replaced by an UwizBernoulliDropout-Layer
#   This allows the uncertainty wizard to control randomness during inference.
print(model.inner.summary())


**Step 3: Make predictions and get the uncertainties and confidences**

In [1]:
quantifiers = ['pcs', 'mean_softmax']
results = model.predict_quantified(x_test,
                                   quantifier=quantifiers,
                                   batch_size=64,
                                   sample_size=32,
                                   verbose=1)

# results[0][0] contains the point-predictions from the 'pcs' quantifier
#     (i.e., argmax of a single non-randomized network forward pass)
# results[0][1] contains the prediction confidence scores for these predictions

# results[1][0] contains the predictions from the 'mean_softmax' quantifier
#     (i.e., argmax of 32 averages forward pass samples on the randomized DNN)
# results[1][1] contains the corresponding confidence
#     (i.e., the average softmax value of the class with the highest average softmax value)

NameError: name 'model' is not defined